# Pandas Run

In [12]:
import os
import pandas as pd
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [13]:
# PATH_DATA = '../../df-showdown-data/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output/'

SLA_NAME = 'pandas-sla.pkl'
MEM_NAME = 'pandas-mem.pkl'


In [14]:
all_sla = {}
all_memory = {}

In [15]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [16]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [17]:
for i in range(6):
    print(i)
    fn = 'data_{}.csv'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = pd.read_csv(PATH_DATA+fn, memory_map=True)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['csv'][nrows]=end
    all_memory['task1']['csv'][nrows]=end_mem

    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.to_csv(TEMP_PATH + 'temp.csv',index=False)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['csv'][nrows]=end
    all_memory['task2']['csv'][nrows]=end_mem

    del dtemp1
    gc.collect()
    os.remove(TEMP_PATH + 'temp.csv')
    time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = pd.read_parquet(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    

    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.to_parquet(TEMP_PATH + 'temp.parquet',index=False)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

In [18]:
all_sla

{'task1': {'csv': {'0': 0.17180899999993926,
   '1': 0.9694262999996681,
   '5': 5.645235400000274},
  'prq': {'0': 0.1768523999999161,
   '1': 0.34727029999976367,
   '5': 1.5775460999998359}},
 'task2': {'csv': {'0': 0.4915897999999288,
   '1': 3.543433499999992,
   '5': 17.182294600000205},
  'prq': {'0': 0.10942849999992177,
   '1': 0.6363883000003625,
   '5': 2.872138900000209}}}

In [19]:
all_memory

{'task1': {'csv': {'0': 0.011894784000000769,
   '1': 0.12227379199999966,
   '5': 0.5949071359999998},
  'prq': {'0': 0.05695897600000066,
   '1': 0.1579294719999993,
   '5': 0.5268725759999988}},
 'task2': {'csv': {'0': 0.0011755519999994135,
   '1': -0.02192179200000055,
   '5': 0.004956159999998988},
  'prq': {'0': 0.03245260799999983,
   '1': 0.07759052799999999,
   '5': 0.07286784000000068}}}

## Get Data for other task

In [ ]:
all_df = [pd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7]]
df_right = pd.read_parquet(PATH_DATA+'data_to_join.parquet')

## Task3 - Sorting

In [ ]:
n_task = 'task3'
all_sla[n_task] = {}

In [ ]:
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=[True,False])
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)


In [ ]:
all_sla[n_task]

{'3': 0.63470099999995,
 '9': 2.022421900000154,
 '19': 4.136806799999931,
 '34': 10.888667199999873,
 '45': 16.219327799999974}

## Task4 - Filtering

In [ ]:
n_task = 'task4'
all_sla[n_task] = {}

In [ ]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)


In [ ]:
all_sla[n_task]

{'3': 0.21421140000006744,
 '9': 0.5953160999999909,
 '19': 1.2111443000001145,
 '34': 2.2449423999999,
 '45': 2.9474488999999267}

## Task5 - Merging

In [ ]:
n_task = 'task5'
all_sla[n_task] = {}

In [ ]:
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = pd.merge(dtemp1,df_right,on=['Date','ship-service-level'],how='left')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 0.5124172000000726,
 '9': 1.4778619999997318,
 '19': 2.9784222000002956,
 '34': 5.352248900000177,
 '45': 7.045791199999712}

## Task6 - udf apply

In [ ]:
n_task = 'task6'
all_sla[n_task] = {}

In [ ]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [ ]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 15.072074999999586,
 '9': 43.90379790000043,
 '19': 86.94100830000025,
 '34': 156.61809700000003,
 '45': 202.96429179999996}

## Task7 - aggregation

In [ ]:
n_task = 'task7'
all_sla[n_task] = {}

In [ ]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [ ]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    gc.collect()
    time.sleep(1)

In [ ]:
all_sla[n_task]

{'3': 0.787891700000273,
 '9': 2.2168736999997236,
 '19': 4.148283800000172,
 '34': 7.475733299999774,
 '45': 9.774048400000083}

## Save results

In [ ]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/pandas-sla.pkl']

In [10]:
# all_sla